In [126]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

load_dotenv()

oapi_key=os.getenv("OPENAI_API_KEY")
api_key=os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)


In [127]:
from langchain_openai import ChatOpenAI

openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)

In [128]:
from langchain_community.document_loaders import PyPDFLoader

namespace = "resume2"
data_applicant = (
    {
        "name": "sarah",
        "file_path": "./dummy-cv/sarah-highly_accurate.pdf",
        "gender": "female",
    },
    {
        "name": "michael",
        "file_path": "./dummy-cv/michael-highly_accurate.pdf",
        "gender": "male",
    },
    {
        "name": "emily",
        "file_path": "./dummy-cv/emily-somewhat-relevant.pdf",
        "gender": "female",
    },
    {
        "name": "david",
        "file_path": "./dummy-cv/david-less-relevant.pdf",
        "gender": "male",
    },
    {
        "name": "anna",
        "file_path": "./dummy-cv/anna-irelevant.pdf",
        "gender": "female"
    },
    {
        "name": "fathur",
        "file_path": "./dummy-cv/sample-cv.pdf",
        "gender": "male"
    }
)

In [129]:
from langchain_pinecone import PineconeVectorStore, PineconeEmbeddings
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=oapi_key,
)

example_question = {
    "1": "What is your name?",
    "2": "What is your email address?",
    "3": "What is your phone number?",
    "4": "Do you have any work experience in TypeScript development? If yes, please describe.",
    "5": "How long have you been working with TypeScript?",
    "6": "What is your experience with TypeScript frameworks like NestJS or Next.js?",
    "7": "Have you ever worked with type definitions (`.d.ts` files) in TypeScript? If yes, can you provide an example?",
    "8": "Do you have experience with JavaScript/TypeScript build tools like Webpack or Vite?",
    "9": "Do you have any experience integrating TypeScript with backend technologies such as Node.js?",
    "10": "Have you worked with TypeScript in a large-scale project? If so, what was your role and what challenges did you face?"
}

In [130]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""Read the following text and answer the question, if you don't know the answer, just fill null. Returning a JSON object with an key for each question and the answer as the value. text: {text}.question: {question}""")
json_parser = SimpleJsonOutputParser()

json_chain = prompt | llm | json_parser

In [131]:
prompt_score = PromptTemplate.from_template(
    """
        You are provided with a job description, a text (which includes a question and the candidate's answer), and your task is to evaluate the relevance of the candidate's response based on the job description. Your goal is to return a JSON object that contains:
        'summary': A brief summary of the candidate's answer.
        'reason': Justification for the relevance score based on the job description.
        'score': A relevance score for the candidate’s response to the job description. The scoring scale is:
            0-20: Not relevant
            21-40: Less relevant
            41-80: Relevant
            81-100: Perfectly relevant
        
        job_description: {job_description}
        text: {text}
        question: {question}
    """
)

score_chain = prompt_score | llm | json_parser

py_desc= 'Job Title: Python Developer. Overview: We are looking for a Python Developer to join our team to design, develop, and maintain scalable applications and APIs. You will collaborate closely with other developers, product managers, and stakeholders to ensure high-quality deliverables. Responsibilities: Develop and maintain Python-based applications, build and maintain RESTful APIs, debug and troubleshoot software bugs, optimize performance, write clean and maintainable code, collaborate on software architecture, conduct code reviews, and assist in writing test cases. Qualifications: Strong proficiency in Python and familiarity with its libraries and frameworks (e.g., Django, Flask), understanding of RESTful services and API integrations, experience with version control systems like Git, excellent problem-solving and debugging skills, and experience working in an Agile environment. Nice-to-Have: Experience with asynchronous programming (e.g., asyncio, Celery), familiarity with cloud services like AWS, Google Cloud, or Azure, experience with CI/CD pipelines, and proficiency with testing tools such as Pytest or Unittest. How to Apply: Submit your application through our website or via [Application Link]; only shortlisted candidates will be contacted.'
job_description = "Job Title: TypeScript Developer. Overview: We are looking for a TypeScript Developer to join our team. You will work on building and maintaining web applications, collaborating with other developers to deliver high-quality software. This is a great opportunity to work in a dynamic environment and grow your skills. Responsibilities: Develop and maintain applications using TypeScript. Build and maintain APIs. Troubleshoot and fix software bugs. Collaborate with team members to deliver projects. Participate in code reviews and testing. Qualifications: Experience with TypeScript and JavaScript. Familiarity with modern front-end frameworks like React, Angular, or Vue. Understanding of RESTful APIs. Experience with version control (Git). Good problem-solving skills. Nice-to-Have: Experience with Node.js or backend development. Familiarity with cloud services (AWS, Azure). Experience with testing tools like Jest. How to Apply: Submit your application through our website or via [Application Link]. Only selected candidates will be contacted."
sr_desc='Job Title: Senior TypeScript Developer. Overview: We are looking for a Senior TypeScript Developer to lead the development of high-quality web applications and mentor the development team. Responsibilities: Design and build scalable applications using TypeScript, lead architectural decisions, oversee code quality, guide best practices, build and maintain RESTful APIs, debug complex issues, and conduct code reviews. Qualifications: Extensive experience with TypeScript and modern front-end frameworks (React, Angular, Vue), deep understanding of RESTful APIs, strong expertise in version control (Git), and proven leadership skills. Nice-to-Have: Experience with Node.js, cloud services (AWS, Azure), and advanced testing tools (Jest, Cypress). How to Apply: Apply through [Application Link]; only selected candidates will be contacted.'
kt_desc="Job Title: Kotlin Developer. Overview: We are seeking a Kotlin Developer to design, develop, and maintain robust applications and services. Responsibilities: Build and maintain applications using Kotlin, develop and integrate APIs, troubleshoot and resolve issues, contribute to code reviews, and collaborate on software design and architecture. Qualifications: Strong experience with Kotlin and Android development, knowledge of modern software development practices, experience with RESTful APIs, and proficiency in version control (Git). Nice-to-Have: Experience with Spring Boot, cloud platforms (AWS, GCP, Azure), and testing frameworks (JUnit, Espresso). How to Apply: Submit your application via [Application Link]; only shortlisted candidates will be contacted."


In [132]:
import sqlite3

con = sqlite3.connect("rag.db")
cur = con.cursor()

create_table_query = '''
CREATE TABLE IF NOT EXISTS user (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    firstname TEXT NOT NULL,
    lastname TEXT NOT NULL DEFAULT '',
    gender TEXT NOT NULL,
    age INTEGER NOT NULL
);

CREATE TABLE IF NOT EXISTS job (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT NOT NULL DEFAULT '',
    location TEXT NOT NULL,
    salary INTEGER NOT NULL
);

CREATE TABLE IF NOT EXISTS job_applicant (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    score INTEGER NOT NULL,
    ai_summary TEXT NOT NULL DEFAULT '',
    user_ID INTEGER NOT NULL,
    job_ID INTEGER NOT NULL,

    FOREIGN KEY (user_ID) REFERENCES user(id),
    FOREIGN KEY (job_ID) REFERENCES job(id)
);
'''

cur.executescript(create_table_query)

In [133]:
data_job = [
    ('Typescript developer', job_description, 'bsd', 10000),
    ('Python developer', py_desc, 'jakarta', 15000),
    ('Senior typescript developer', sr_desc, 'bogor', 17000),
    ('Kotlin developer', kt_desc, 'tanggerang', 20000)
]

cur.executemany("INSERT INTO job (title, description, location, salary) VALUES (?,?,?,?)", data_job)
con.commit()

In [134]:
res = cur.execute("SELECT * FROM job WHERE title LIKE '%Typescript developer%'").fetchone()

In [135]:
job_id, title, job_desc, location, salary = res

In [136]:
from langchain_core.documents import Document

index_name = "applicants-2"
openai_embedding_small_3_dimentions = 1536

pc.create_index(
    name=index_name,
    dimension=openai_embedding_small_3_dimentions,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

for x in data_applicant:
    loader = PyPDFLoader(x["file_path"], extract_images=True)
    pages = loader.load()

    doc: list[Document]= []

    result = json_chain.invoke({
        "text": list(map(lambda x: x.page_content, pages)),
        "question": example_question
    })

    result_scoring = score_chain.invoke({
        "text": list(map(lambda x: x.page_content, pages)),
        "question": result,
        "job_description": job_desc
    })

    for y in pages:
        doc.append(
            Document(y.page_content, metadata={ "gender": x["gender"], "name": x["name"], "score": result_scoring["score"], "reason": result_scoring["reason"], "summary": result_scoring["summary"]})
        )

    data_user = (x["name"], "lastname", x["gender"], 10)

    cur.execute("INSERT INTO user(firstname,lastname, gender, age) VALUES(?, ?, ?, ?)", data_user)
    user_id = cur.lastrowid

    data_job_app = (result_scoring["score"], result_scoring["summary"], user_id, job_id)
    cur.execute("INSERT INTO job_applicant(score, ai_summary, user_ID, job_ID) VALUES(?, ?, ?, ?)", data_job_app)

    docsearch = PineconeVectorStore.from_documents(
        embedding=embedding,
        index_name=index_name,
        documents=doc,
        namespace=namespace
    )


In [137]:
# import click

# def pprompt(question: str, default: str) -> None:
#     res = click.prompt(text=f"{question}\n", default=value if value != "null" else "" ,type=str)
#     if res == "":
#        return pprompt(question, default)
    
#     return res

# for key, value in result.items():
#     res: str = pprompt(example_question[key], default=value)
#     result[key] = res



In [138]:
query = "i need candidate that have experience on typescript but Prefer for fullstack engineer that can lead small teams"

vectors = embedding.embed_query(query)
index = pc.Index(name=index_name)

qr = index.query(
    namespace=namespace, 
    top_k=5,
    filter={
        "score": {"$gt": 30},
    },
    vector=vectors,
    include_values=False,
    include_metadata=True
)


In [139]:
qr

{'matches': [{'id': 'c9da28b3-707e-436a-9312-a7d004a3da73',
              'metadata': {'gender': 'female',
                           'name': 'sarah',
                           'reason': "The candidate's experience aligns "
                                     'closely with the job description '
                                     'requirements, including extensive work '
                                     'with TypeScript and JavaScript, '
                                     'experience in building and maintaining '
                                     'APIs, and familiarity with modern '
                                     'front-end frameworks like React and '
                                     'Angular. Additionally, her background in '
                                     'leading teams and working on cloud-based '
                                     'applications demonstrates strong '
                                     'collaboration and problem-solving '
             

In [140]:
con.commit()

In [145]:
applicants = cur.execute("SELECT * FROM job_applicant j1 LEFT JOIN user u1 ON j1.user_ID = u1.id").fetchall()

In [146]:
applicants

[(1,
  95,
  'Sarah Johnson has over 6 years of experience as a Senior TypeScript Developer and Full-Stack Developer, specializing in building applications with TypeScript, React, and Node.js. She has led teams and worked on cloud-based applications and RESTful APIs, focusing on code quality and scalable architecture.',
  1,
  1,
  1,
  'sarah',
  'lastname',
  'female',
  10),
 (2,
  95,
  'Michael Lee has 5 years of experience in TypeScript and JavaScript development, focusing on building and maintaining applications, including a multi-tenant SaaS application using TypeScript, React, and Node.js. He has also worked on REST API design and automated testing procedures.',
  2,
  1,
  2,
  'michael',
  'lastname',
  'male',
  10),
 (3,
  61,
  'Emily has approximately 1 year of experience with TypeScript, primarily through a project management tool, and has integrated it with backend technologies like Node.js.',
  3,
  1,
  3,
  'emily',
  'lastname',
  'female',
  10),
 (4,
  20,
  'Dav